# OmniSafe Tutorial - Environment Customization from Community

OmniSafe: https://github.com/PKU-Alignment/omnisafe

Documentation: https://omnisafe.readthedocs.io/en/latest/

Gymnasium: https://github.com/Farama-Foundation/Gymnasium

[Gymnasium](https://github.com/Farama-Foundation/Gymnasium) is an open source Python library for developing and comparing reinforcement learning algorithms by providing a standard API to communicate between learning algorithms and environments, as well as a standard set of environments compliant with that API.

## Introduction

In this section, we will introduce how to embed an existing environment from the community into OmniSafe. The series of tasks provided by [Gymnasium](https://github.com/Farama-Foundation/Gymnasium) have been widely applied in reinforcement learning. Specifically, this section will use [Pendulum-v1](https://gymnasium.farama.org/environments/classic_control/pendulum/) as an example to show how to embed Gymnasium's tasks into OmniSafe.

## Quick Installation

In [ ]:
# Install via pip (ignore it if you have already installed).
%pip install omnisafe

In [ ]:
# Install from source (ignore it if you have already installed).
## clone the repo
%git clone https://github.com/PKU-Alignment/omnisafe
%cd omnisafe

## install it
%pip install -e .

## Gymnasium Task Embedding
The core part for environment embedding is to provide sufficient static or dynamic information for SafeRL agent interaction and training. This section will detail the variables that must be defined for embedding environments and the corresponding standards. We will first present the entire embedding process in the order of code organization, giving a preliminary understanding. Then, we will review all the codes, summarize, and organize the adaptations you need to make when customizing your environment.

### Quick Start
First, import all external variables required for this tutorial.

In [ ]:
# import all we need
from __future__ import annotations

from typing import Any, ClassVar
import gymnasium
import torch
import numpy as np
import omnisafe

from omnisafe.envs.core import CMDP, env_register, env_unregister
from omnisafe.typing import DEVICE_CPU

Next, create a class named `ExampleMuJoCoEnv`, which needs to inherit from `CMDP`. (This is because we want to transform the environment's interaction form into the CMDP paradigm. You can define new abstract classes as needed to implement new paradigms).

In [ ]:
class ExampleMuJoCoEnv(CMDP):
    _support_envs: ClassVar[list[str]] = ['Pendulum-v1']  # Supported task names

    need_auto_reset_wrapper = True  # Whether `AutoReset` Wrapper is needed
    need_time_limit_wrapper = True  # Whether `TimeLimit` Wrapper is needed

    def __init__(
        self,
        env_id: str,
        num_envs: int = 1,
        device: torch.device = DEVICE_CPU,
        **kwargs: Any,
    ) -> None:
        super().__init__(env_id)
        self._num_envs = num_envs
        # Instantiate the environment object
        self._env = gymnasium.make(id=env_id, autoreset=True, **kwargs)
        # Specify the action space for initialization by the algorithm layer
        self._action_space = self._env.action_space
        # Specify the observation space for initialization by the algorithm layer
        self._observation_space = self._env.observation_space
        # Optional, for GPU acceleration. Default is CPU
        self._device = device  # 可选项，使用GPU加速。默认为CPU

    def reset(
        self,
        seed: int | None = None,
        options: dict[str, Any] | None = None,
    ) -> tuple[torch.Tensor, dict[str, Any]]:
        # Reset the environment
        obs, info = self._env.reset(seed=seed, options=options)
        # Convert the reset observations to a torch tensor.
        return (
            torch.as_tensor(obs, dtype=torch.float32, device=self._device),
            info,
        )

    @property
    def max_episode_steps(self) -> int | None:
        # Return the maximum number of interaction steps per episode in the environment
        return self._env.env.spec.max_episode_steps

    def set_seed(self, seed: int) -> None:
        # Set the environment's random seed for reproducibility
        self.reset(seed=seed)  # 设定环境的随机种子以实现可复现性

    def render(self) -> Any:
        # Return the image rendered by the environment
        return self._env.render()

    def close(self) -> None:
        # Release the environment instance after training ends
        self._env.close()

    def step(
        self,
        action: torch.Tensor,
    ) -> tuple[
        torch.Tensor,
        torch.Tensor,
        torch.Tensor,
        torch.Tensor,
        torch.Tensor,
        dict[str, Any],
    ]:
        # Read the dynamic information after interacting with the environment
        obs, reward, terminated, truncated, info = self._env.step(
            action.detach().cpu().numpy(),
        )
        # Gymnasium does not explicitly include safety constraints; this is just a placeholder.
        cost = np.zeros_like(reward)
        # Convert dynamic information into torch tensor.
        obs, reward, cost, terminated, truncated = (
            torch.as_tensor(x, dtype=torch.float32, device=self._device)
            for x in (obs, reward, cost, terminated, truncated)
        )
        if 'final_observation' in info:
            info['final_observation'] = np.array(
                [
                    array if array is not None else np.zeros(obs.shape[-1])
                    for array in info['final_observation']
                ],
            )
            # Convert the last observation recorded in info into a torch tensor.
            info['final_observation'] = torch.as_tensor(
                info['final_observation'],
                dtype=torch.float32,
                device=self._device,
            )

        return obs, reward, cost, terminated, truncated, info

Regarding the specific meaning of the above code, we have provided detailed annotation explanations. For more detailed explanations, please refer to [Tutorial 3: Environment Customization from Zero](./3.Environment%20Customization.ipynb). We summarize the key points as follows:

- **Static variables needed for OmniSafe initialization**

| Static Information | Required | Definition | Type | Example |
|:---:|:---:|:---:|:---:|:---:|
| `need_auto_reset_wrapper` | Yes | Whether an `AutoReset` Wrapper is needed | `bool` variable | `True` |
| `need_time_limit_wrapper` | Yes | Whether a `TimeLimit` Wrapper is needed | `bool` variable | `True` |
| `_action_space` | Yes | Action space | `gymnasium.space.Box` | `Box(low=-1.0, high=1.0, shape=(2,)` |
| `_observation_space` | Yes | Observation space | `gymnasium.space.Box` | `Box(low=-1.0, high=1.0, shape=(3,)` |
| `max_episode_steps` | Yes | The maximum number of interaction steps per episode in the environment | Function with `@property` decorator, returning a variable of type `int` or `None` | Refer to the code block above |
| `_num_envs` | No | Number of parallel environments | `int` variable | 5 |
| `_device` | No | Torch computing device | `torch.device` variable | `DEVICE_CPU` |

- **Dynamic variables required by the environment for OmniSafe**

OmniSafe's agents mainly interact dynamically with the environment through the `reset` and `step` functions. You need to ensure that the return type, number, and order of your customized environment match the examples above, more specifically:

| Dynamic Information | Type | Number | Order |
|:---:|:---:|:---:|:---:|
| `step` | `tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, dict[str, Any]]` | 6 | `obs`, `reward`, `cost`, `terminated`, `truncated`, `info` |
| `reset` | `tuple[torch.Tensor, dict[str, Any]]` | 2 | `obs`, `info` |

- **Precautions**

1. Although `_num_envs` and `_device` are not mandatory, please retain the input interface for these two parameters in the `__init__` function.
2. `_num_envs` is an advanced parameter for instantiating multiple environments for parallel sampling, representing the number of environments instantiated. If your customized environment also supports specifying the parallel number, please specify it through `_num_envs` instead of defining a new interface.

Subsequently, by registering the above environment into OmniSafe with the registration decorator `@env_register`, you can complete the training.

In [ ]:
@env_register
@env_unregister  # Avoid the "environment has been registered" error when rerunning cells
class ExampleMuJoCoEnv(ExampleMuJoCoEnv):
    pass


custom_cfgs = {
    'train_cfgs': {
        'total_steps': 200,
    },
    'algo_cfgs': {
        'steps_per_epoch': 200,
        'update_iters': 1,
    },
}
agent = omnisafe.Agent('PPOLag', 'Pendulum-v1', custom_cfgs=custom_cfgs)
agent.learn()

### Advanced Usage
In addition to the aforementioned methods, environments from the community can also take advantage of OmniSafe's capabilities for specifying environment-specific parameters and recording information. We will detail the specific operational methods.

#### Specifying Specific Parameters

Taking `Pendulum-v1` as an example, according to the Gymnasium documentation, a specific parameter `g`, which stands for gravitational acceleration, can be specified when creating this task. Let's first take a look at its default value:

In [ ]:
@env_register
@env_unregister  # Avoid the "environment has been registered" error when rerunning cells
class ExampleMuJoCoEnv(ExampleMuJoCoEnv):
    def __getattr__(self, name: str) -> Any:
        """Get the attribute of the environment."""
        if name.startswith('_'):
            raise AttributeError(f'attempted to get missing private attribute {name}')
        return getattr(self._env, name)


custom_cfgs = {
    'train_cfgs': {
        'total_steps': 200,
    },
    'algo_cfgs': {
        'steps_per_epoch': 200,
        'update_iters': 1,
    },
}
agent = omnisafe.Agent('PPOLag', 'Pendulum-v1', custom_cfgs=custom_cfgs)
agent.agent._env._env.g

We implemented a magic function named `__get_attr__` to call and view specific parameters in the currently instantiated environment. In this case, we find that the default value of the gravitational acceleration `g` is 10.0.

By consulting the Gymnasium documentation, this parameter can be specified during the process of creating an environment with the `gymnasium.make` function. Does OmniSafe support the passing of specific parameters for customized environments? The answer is yes:

In [ ]:
custom_cfgs.update({'env_cfgs': {'g': 9.8}})
agent = omnisafe.Agent('PPOLag', 'Pendulum-v1', custom_cfgs=custom_cfgs)
agent.agent._env._env.g

Nice! The value of gravitational acceleration has been changed to 9.8. We just need to operate on `env_cfgs`, specifying the key and value of the parameter to be customized, to achieve the passing of specific parameters for the environment.

#### Information Recording

The `Pendulum-v1` task contains many specific dynamic pieces of information. We will introduce how to record these pieces of information using OmniSafe's `Logger`. Specifically, we will explain using the maximum and cumulative values of the angular velocity `angular_velocity` per episode as examples.

In [ ]:
from omnisafe.common.logger import Logger


@env_register
@env_unregister  # Avoid the "environment has been registered" error when rerunning cells
class ExampleMuJoCoEnv(ExampleMuJoCoEnv):

    def __init__(self, env_id, num_envs, device, **kwargs):
        super().__init__(env_id, num_envs, device, **kwargs)
        self.env_spec_log = {
            'Env/Max_angular_velocity': 0.0,
            'Env/Cumulative_angular_velocity': 0.0,
        }  # Reiterate and specify in the constructor

    def spec_log(self, logger: Logger) -> None:
        for key, value in self.env_spec_log.items():
            logger.store({key: value})
            self.env_spec_log[key] = 0.0

    def step(self, action):
        obs, reward, cost, terminated, truncated, info = super().step(action=action)
        angle = obs[-1].item()
        self.env_spec_log['Env/Max_angular_velocity'] = max(
            self.env_spec_log['Env/Max_angular_velocity'], angle
        )
        self.env_spec_log['Env/Cumulative_angular_velocity'] += angle
        return obs, reward, cost, terminated, truncated, info


agent = omnisafe.Agent('PPOLag', 'Pendulum-v1', custom_cfgs=custom_cfgs)
agent.learn()

Great! We successfully recorded the required environment-specific information in the `Logger`. It is worth noting that, in this process, we did not modify any of OmniSafe's source code.

## Summary
In this section, using Gymnasium's classic environment `Pendulum-v1`, we introduced the necessary interface adaptation and information provision required to embed an existing community environment into OmniSafe. We hope this tutorial is helpful for embedding your customized environment. If you wish to have your environment supported as one of the official OmniSafe environments, or if you encounter difficulties in customizing environments, you are welcome to communicate with us through the [Issues](https://github.com/PKU-Alignment/omnisafe/issues), [Pull Requests](https://github.com/PKU-Alignment/omnisafe/pulls), and [Discussions](https://github.com/PKU-Alignment/omnisafe/discussions) modules.